In [2]:
import pandas as pd

# prevent false warning
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import datetime as dt

from decouple import Config, RepositoryEnv

DOTENV_FILE = '../.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))


In [3]:
#Downloading weather data using Python as a CSV using the Visual Crossing Weather API
#See https://www.visualcrossing.com/resources/blog/how-to-load-historical-weather-data-using-python-without-scraping/ for more information.
import csv
import codecs
import urllib.request
import urllib.error
import sys

# This is the core of our weather query URL
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

ApiKey=env_config.get('VISUALCROSSING_API_KEY')
#UnitGroup sets the units of the output - us or metric
UnitGroup='metric'

#Location for the weather data
Location='kigoma,tz'

#Optional start and end dates
#If nothing is specified, the forecast is retrieved. 
#If start date only is specified, a single historical or forecast day will be retrieved
#If both start and and end date are specified, a date range will be retrieved
StartDate = '2018-01-01'
EndDate='2018-01-02'

#JSON or CSV 
#JSON format supports daily, hourly, current conditions, weather alerts and events in a single JSON package
#CSV format requires an 'include' parameter below to indicate which table section is required
ContentType="csv"

#include sections
#values include days,hours,current,alerts
Include="days"


print('')
print(' - Requesting weather : ')

#basic query including location
ApiQuery=BaseURL + Location

#append the start and end date if present
if (len(StartDate)):
    ApiQuery+="/"+StartDate
    if (len(EndDate)):
        ApiQuery+="/"+EndDate

#Url is completed. Now add query parameters (could be passed as GET or POST)
ApiQuery+="?"

#append each parameter as necessary
if (len(UnitGroup)):
    ApiQuery+="&unitGroup="+UnitGroup

if (len(ContentType)):
    ApiQuery+="&contentType="+ContentType

if (len(Include)):
    ApiQuery+="&include="+Include

ApiQuery+="&key="+ApiKey



print(' - Running query URL: ', ApiQuery)
print()

try: 
    CSVBytes = urllib.request.urlopen(ApiQuery)
except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()
except  urllib.error.URLError as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code,ErrorInfo)
    sys.exit()


# Parse the results as CSV
CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

RowIndex = 0

# The first row contain the headers and the additional rows each contain the weather metrics for a single day
# To simply our code, we use the knowledge that column 0 contains the location and column 1 contains the date.  The data starts at column 4
for Row in CSVText:
    if RowIndex == 0:
        FirstRow = Row
    else:
        print('Weather in ', Row[0], ' on ', Row[1])

        ColIndex = 0
        for Col in Row:
            if ColIndex >= 4:
                print('   ', FirstRow[ColIndex], ' = ', Row[ColIndex])
            ColIndex += 1
    RowIndex += 1

# If there are no CSV rows then something fundamental went wrong
if RowIndex == 0:
    print('Sorry, but it appears that there was an error connecting to the weather server.')
    print('Please check your network connection and try again..')

# If there is only one CSV  row then we likely got an error from the server
if RowIndex == 1:
    print('Sorry, but it appears that there was an error retrieving the weather data.')
    print('Error: ', FirstRow)

print()


 - Requesting weather : 
 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/kigoma,tz/2018-01-01/2018-01-14?&unitGroup=metric&contentType=csv&include=days&key=PRYLRKJQZXXEK4YCXLLNEW525

Weather in  Kigoma, Tanzania  on  2018-01-01
    temp  =  22.9
    feelslikemax  =  29.3
    feelslikemin  =  20.5
    feelslike  =  23.1
    dew  =  20.5
    humidity  =  87
    precip  =  9.3
    precipprob  =  100
    precipcover  =  79.17
    preciptype  =  rain
    snow  =  
    snowdepth  =  
    windgust  =  
    windspeed  =  25.9
    winddir  =  211.5
    sealevelpressure  =  
    cloudcover  =  69.2
    visibility  =  16.2
    solarradiation  =  170
    solarenergy  =  14.7
    uvindex  =  6
    severerisk  =  
    sunrise  =  2018-01-01T06:52:57
    sunset  =  2018-01-01T19:16:41
    moonphase  =  0.5
    conditions  =  Rain, Partially cloudy
    description  =  Partly cloudy throughout the day with a chance of rain throughout the day.


In [6]:
df = pd.DataFrame(CSVText)

In [12]:
with open('employee_file2.csv', mode='w') as csv_file:
    first = False
    fieldnames = None

    for Row in CSVText:
        print(Row)
        fieldnames = Row
        break
        
    print(fieldnames)
        
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for Row in CSVText:
        
        if not first:
            first = True
            continue
        
        writer.writerow(Row)
    

None


TypeError: 'NoneType' object is not iterable

In [23]:
for row in CSVText:
    print(x)
    print(row)

In [20]:
CSVBytes.read().content

AttributeError: 'bytes' object has no attribute 'content'